<a href="https://colab.research.google.com/github/Netoperek5000/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [13]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout 
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )
  
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape), 
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(), 
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(), 
    Dropout(0.3),

    Flatten(),  

    Dense(1024, activation='relu'), 
    Dropout(0.3),
  
    Dense(1024, activation='relu'), 
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),             
  ])

In [9]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 7s 25ms/step - loss: 2.6541 - accuracy: 0.2835 - val_loss: 1.3085 - val_accuracy: 0.5765
Epoch 2/5
272/272 [==============================] - 6s 23ms/step - loss: 1.0466 - accuracy: 0.6627 - val_loss: 0.2985 - val_accuracy: 0.9044
Epoch 3/5
272/272 [==============================] - 6s 23ms/step - loss: 0.5402 - accuracy: 0.8296 - val_loss: 0.1513 - val_accuracy: 0.9551
Epoch 4/5
272/272 [==============================] - 6s 23ms/step - loss: 0.3445 - accuracy: 0.8933 - val_loss: 0.0947 - val_accuracy: 0.9739
Epoch 5/5
272/272 [==============================] - 6s 23ms/step - loss: 0.2603 - accuracy: 0.9213 - val_loss: 0.0465 - val_accuracy: 0.9852


0.9616780045351474

In [10]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 4ms/step - loss: 0.1247 - accuracy: 0.9617


[0.12474961578845978, 0.9616780281066895]

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape), 
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(), 
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(), 
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),  

    Dense(1024, activation='relu'), 
    Dropout(params['dropout_dense_block_one']),
  
    Dense(1024, activation='relu'), 
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),             
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [27]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 150.0, 'dropout_cnn_block_one': 0.4471543062092206, 'dropout_cnn_block_three': 0.4582027256417376, 'dropout_cnn_block_two': 0.3713947604103903, 'dropout_dense_block_one': 0.30091564772917284, 'dropout_dense_block_two': 0.6067158249750717}
accuracy=0.940816342830658
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.354286406870632, 'dropout_cnn_block_three': 0.42225514676881554, 'dropout_cnn_block_two': 0.3113679549458579, 'dropout_dense_block_one': 0.39938512217711497, 'dropout_dense_block_two': 0.5438298369512653}
accuracy=0.9448979496955872
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.45452677694609755, 'dropout_cnn_block_three': 0.4732619684377231, 'dropout_cnn_block_two': 0.49628518579628284, 'dropout_dense_block_one': 0.5542635285195918, 'dropout_dense_block_two': 0.6088909897433579}
accuracy=0.8662131428718567
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.36355992195252096, 'dropout_cnn_block_three': 0.3583355226795975, 'dropout_cnn_block_two': 0.304941636306